In [ ]:
# import das biblioteca
import gurobipy as gp
from gurobipy import GRB

import pandas as pd
import numpy as np

In [ ]:
# cria o modelo
m = gp.Model()

# variáveis
x1 = m.addVar(obj=2, name="x1")
x2 = m.addVar(obj=4, name="x2")
x3 = m.addVar(obj=1, name="x3")

# restrições
rest1 = m.addConstr(2*x1 + x2 + x3 <= 10, "restricao1")
rest2 = m.addConstr(x1 + x2 - x3 <= 4, "restricao2")
rest3 = m.addConstr(x1 <= 4, "restricao3")
rest4 = m.addConstr(x2 <= 6, "restricao4")
rest5 = m.addConstr(x3 <= 6, "restricao5")
rest6 = m.addConstr(x3 >= 1, "restricao6")

# definindo funcao objetivo
m.setObjective(2*x1 + 4*x2 + x3, GRB.MAXIMIZE)

# escrevendo .lp/.mps
m.write('exemplo.lp')

# resolve o problema
m.optimize()

# imprime a solução ótima
for v in m.getVars():
    print(v.varName, v.x)
    
# imprime o valor ótimo
print('opt : ', m.objVal)

In [ ]:
# problema mestre

# cria o modelo
mestre = gp.Model()

# variáveis
l1 = mestre.addVar(name="l1")
l2 = mestre.addVar(name="l2")
l3 = mestre.addVar(name="l3")
l4 = mestre.addVar(name="l4")
l5 = mestre.addVar(name="l5")
l6 = mestre.addVar(name="l6")
l7 = mestre.addVar(name="l7")
l8 = mestre.addVar(name="l8")

# restrições
rest1 = mestre.addConstr(9*l1 + 20*l2 + 12*l3 + 7*l4 + 9*l5 + 14*l6 + 6*l7 + l8 <= 10, "restricao1")
rest2 = mestre.addConstr(3*l1 + 4*l2 + 0*l3 + 5*l4 + 3*l5 + (-2)*l6 + (-6)*l7 + (-1)*l8 <= 4, "restricao2")

# definindo funcao objetivo
mestre.setObjective(9*l1 + 38*l2 + 30*l3 + 25*l4 + 9*l5 + 14*l6 + 6*l7 + 1*l8, GRB.MAXIMIZE)

# escrevendo .lp/.mps
mestre.write('mestre.lp')

# resolve o problema
mestre.optimize()

# imprime a solução ótima
for v in mestre.getVars():
    print(v.varName, v.x)

# imprime o valor ótimo
print('opt mestre: ', mestre.objVal)

In [ ]:
# iteração 1

# problema mestre restrito 1

v1 = np.array([0,0,1])
print("v1 :", v1)

# v^1 = (0,0,1)
# p^1 = A_1 * (0,0,1) = (1,-1)
# f^1 = c * (0,0,1) = 1

# cria o modelo
pmr1 = gp.Model()

# variáveis
l1 = pmr1.addVar(name="l1")

# restrições
rest1 = pmr1.addConstr(l1 <= 10, "restricao1")
rest2 = pmr1.addConstr(-l1 <= 4, "restricao2")
rest3 = pmr1.addConstr(l1 == 1, "restricao3")

# definindo funcao objetivo
pmr1.setObjective(1*l1, GRB.MAXIMIZE)

# escrevendo .lp/.mps
pmr1.write('pmr1.lp')

# resolve o problema
pmr1.optimize()

# imprime a solução ótima
for v in pmr1.getVars():
    print(v.varName, v.x)

# imprime o valor ótimo
print('opt pmr1: ', pmr1.objVal)

# get valores do dual
#for c in pmr1.getConstrs():
#    print ('The dual value of %s : %g' %(c.constrName,c.pi))
    
mi1 = rest1.pi
mi2 = rest2.pi
mi3 = rest3.pi

print("mi1 :", mi1)
print("mi2 :", mi2)
print("mi3 :", mi3)

In [ ]:
# subproblema 1 (price)

subp1 = gp.Model()

# variáveis
x1 = subp1.addVar(obj = 2 - 2*(mi1) - (mi2), name="x1")
x2 = subp1.addVar(obj = 4 - (mi1) - (mi2), name="x2")
x3 = subp1.addVar(obj = 1 - (mi1) + (mi2), name="x3")

# restrições
rest1 = subp1.addConstr(x1 <= 4, "restricao1")
rest2 = subp1.addConstr(x2 <= 6, "restricao2")
rest3 = subp1.addConstr(x3 <= 6, "restricao3")
rest4 = subp1.addConstr(x3 >= 1, "restricao4")

# definindo sentido da funcao objetivo
subp1.modelSense = GRB.MAXIMIZE

# escrevendo .lp/.mps
subp1.write('subp1.lp')

# resolve o problema
subp1.optimize()

# imprime a solução ótima
for v in subp1.getVars():
    print(v.varName, v.x)
    
# imprime o valor ótimo
print('opt subp1: ', subp1.objVal - mi3)

In [ ]:
# limite dual
print('Limite dual: ',  pmr1.objVal + (subp1.objVal - mi3))

In [ ]:
# iteração 2

# problema mestre restrito 2

v2 = np.array([x1.X,x2.X,x3.X])
print("v2 :", v2)

# v^2 = (4,6,6)
# p^2 = A * (4,6,6)
# f^2 = c * (4,6,6)

# cria o modelo
pmr2 = gp.Model()

# variáveis
l1 = pmr2.addVar(obj=1, name="l1")
l2 = pmr2.addVar(obj=38, name="l2")

# restrições
rest1 = pmr2.addConstr(l1 + 20*l2 <= 10, "restricao1")
rest2 = pmr2.addConstr(-l1 + 4*l2 <= 4, "restricao2")
rest3 = pmr2.addConstr(l1 + l2 == 1, "restricao3")

# definindo sentido da funcao objetivo
pmr2.modelSense = GRB.MAXIMIZE

# escrevendo .lp/.mps
pmr2.write('pmr2.lp')

# resolve o problema
pmr2.optimize()

# imprime a solução ótima
for v in pmr2.getVars():
    print(v.varName, v.x)

# imprime o valor ótimo
print('opt pmr2: ', pmr2.objVal)

# get dual
#for c in pmr2.getConstrs():
#    print ('The dual value of %s : %g' %(c.constrName,c.pi))
    
mi1 = rest1.pi
mi2 = rest2.pi
mi3 = rest3.pi

print("mi1 :", mi1)
print("mi2 :", mi2)
print("mi3 :", mi3)

In [ ]:
# subproblema 2 (price)

subp2 = gp.Model()

# variáveis
x1 = subp2.addVar(obj = 2 - 2*(mi1) - (mi2), name="x1")
x2 = subp2.addVar(obj = 4 - (mi1) - (mi2), name="x2")
x3 = subp2.addVar(obj = 1 - (mi1) + (mi2), name="x3")

# restrições/
rest1 = subp2.addConstr(x1 <= 4, "restricao1")
rest2 = subp2.addConstr(x2 <= 6, "restricao2")
rest3 = subp2.addConstr(x3 <= 6, "restricao3")
rest4 = subp2.addConstr(x3 >= 1, "restricao4")

# definindo sentido da funcao objetivo
subp2.modelSense = GRB.MAXIMIZE

# escrevendo .lp/.mps
subp2.write('subp2.lp')

# resolve o problema
subp2.optimize()

# imprime a solução ótima
for v in subp2.getVars():
    print(v.varName, v.x)
    
# imprime o valor ótimo
print('opt subp2: ', (subp2.objVal - mi3))

In [ ]:
# limite dual
print('Limite dual: ',  pmr2.objVal + (subp2.objVal - mi3))

In [ ]:
# iteração 3

# problema mestre restrito 3

v3 = np.array([x1.X, x2.X, x3.X])
print("v3 :", v3)

# v^3 = (0,6,1)
# p^3 = A * (0,6,1)
# f^3 = c * (0,6,1)

# cria o modelo
pmr3 = gp.Model()

# variáveis
l1 = pmr3.addVar(obj=1, name="l1")
l2 = pmr3.addVar(obj=38, name="l2")
l3 = pmr3.addVar(obj=25, name="l3")

# restrições
rest1 = pmr3.addConstr(l1 + 20*l2 + 7*l3 <= 10, "restricao1")
rest2 = pmr3.addConstr(-l1 + 4*l2 + 5*l3<= 4, "restricao2")
rest3 = pmr3.addConstr(l1 + l2 + l3 == 1, "restricao3")

# definindo sentido da funcao objetivo
pmr3.modelSense = GRB.MAXIMIZE

# escrevendo .lp/.mps
pmr3.write('pmr3.lp')

# resolve o problema
pmr3.optimize()

# imprime a solução ótima
for v in pmr3.getVars():
    print(v.varName, v.x)

# imprime o valor ótimo
print('opt pmr3: ', pmr3.objVal)

# get valores do dual
#for c in pmr3.getConstrs():
#    print ('The dual value of %s : %g' %(c.constrName,c.pi))
    
mi1 = rest1.pi
mi2 = rest2.pi
mi3 = rest3.pi

print("mi1 :", mi1)
print("mi2 :", mi2)
print("mi3 :", mi3)

In [ ]:
# subproblema 3 (price)

subp3 = gp.Model()

# variáveis
x1 = subp3.addVar(obj=2 - 2*(mi1) - (mi2), name="x1")
x2 = subp3.addVar(obj=4 - (mi1) - (mi2), name="x2")
x3 = subp3.addVar(obj=1 - (mi1) + (mi2), name="x3")

# restrições
rest1 = subp3.addConstr(x1 <= 4, "restricao1")
rest2 = subp3.addConstr(x2 <= 6, "restricao2")
rest3 = subp3.addConstr(x3 <= 6, "restricao3")
rest4 = subp3.addConstr(x3 >= 1, "restricao4")

# definindo sentido da funcao objetivo
subp3.modelSense = GRB.MAXIMIZE

# escrevendo .lp/.mps
subp3.write('subp3.lp')

# resolve o problema
subp3.optimize()

# imprime a solução ótima
for v in subp3.getVars():
    print(v.varName, v.x)
    
# imprime o valor ótimo
print('opt subp3: ', subp3.objVal - (mi3))

In [ ]:
# limite dual
print('Limite dual: ',  pmr3.objVal + (subp3.objVal - mi3))

In [ ]:
# iteração 4

# problema mestre restrito 4

v4 = np.array([x1.X, x2.X, x3.X])
print("v4 : ", v4)

# v^4 = (0,0,6)
# p^4 = A * (0,0,6)
# f^4 = c * (0,0,6)

# cria o modelo
pmr4 = gp.Model()

# variáveis
l1 = pmr4.addVar(obj=1, name="l1")
l2 = pmr4.addVar(obj=38, name="l2")
l3 = pmr4.addVar(obj=25, name="l3")
l4 = pmr4.addVar(obj=6, name="l4")

# restrições
rest1 = pmr4.addConstr(l1 + 20*l2 + 7*l3 + 6*l4 <= 10, "restricao1")
rest2 = pmr4.addConstr(-l1 + 4*l2 + 5*l3 - 6*l4 <= 4, "restricao2")
rest3 = pmr4.addConstr(l1 + l2 + l3 + l4 == 1, "restricao3")

# definindo sentido da funcao objetivo
pmr4.modelSense = GRB.MAXIMIZE

# escrevendo .lp/.mps
pmr4.write('pmr4.lp')

# resolve o problema
pmr4.optimize()

# imprime a solução ótima
for v in pmr4.getVars():
    print(v.varName, v.x)

# imprime o valor ótimo
print('opt pmr4: ', pmr4.objVal)

# get valores do dual
# for c in pmr4.getConstrs():
#    print ('The dual value of %s : %g' %(c.constrName,c.pi))
    
mi1 = rest1.pi
mi2 = rest2.pi
mi3 = rest3.pi

print("mi1 :", mi1)
print("mi2 :", mi2)
print("mi3 :", mi3)

In [ ]:
# subproblema 4 (price)

subp4 = gp.Model()

# variáveis
x1 = subp4.addVar(obj=2 - 2*(mi1) - (mi2), name="x1")
x2 = subp4.addVar(obj=4 - (mi1) - (mi2), name="x2")
x3 = subp4.addVar(obj=1 - (mi1) + (mi2), name="x3")

# restrições
rest1 = subp4.addConstr(x1 <= 4, "restricao1")
rest2 = subp4.addConstr(x2 <= 6, "restricao2")
rest3 = subp4.addConstr(x3 <= 6, "restricao3")
rest4 = subp4.addConstr(x3 >= 1, "restricao4")

# definindo sentido da funcao objetivo
subp4.modelSense = GRB.MAXIMIZE

# escrevendo .lp/.mps
subp4.write('subp4.lp')

# resolve o problema
subp4.optimize()

# imprime a solução ótima
for v in subp4.getVars():
    print(v.varName, v.x)
    
# imprime o valor ótimo
print('opt subp4: ', subp4.objVal - (mi3))

In [ ]:
# limite dual
print('Limite dual: ',  pmr4.objVal + (subp4.objVal - mi3))

In [ ]:
# iteração 5

# problema mestre restrito 5

v5 = np.array([x1.X, x2.X, x3.X])
print("v5 :", v5)

# problema mestre restrito 5

# v^5 = (0,6,6)
# p^5 = A * (0,6,6)
# f^5 = c * (0,6,6)

# cria o modelo
pmr5 = gp.Model()

# variáveis
l1 = pmr5.addVar(obj=1, name="l1")
l2 = pmr5.addVar(obj=38, name="l2")
l3 = pmr5.addVar(obj=25, name="l3")
l4 = pmr5.addVar(obj=6, name="l4")
l5 = pmr5.addVar(obj=30, name="l5")

# restrições
rest1 = pmr5.addConstr(l1 + 20*l2 + 7*l3 + 6*l4 + 12*l5 <= 10, "restricao1")
rest2 = pmr5.addConstr(-l1 + 4*l2 + 5*l3 - 6*l4 + 0*l5 <= 4, "restricao2")
rest3 = pmr5.addConstr(l1 + l2 + l3 + l4 + l5 == 1, "restricao3")

# definindo sentido da funcao objetivo
pmr5.modelSense = GRB.MAXIMIZE

# escrevendo .lp/.mps
pmr5.write('pmr5.lp')

# resolve o problema
pmr5.optimize()

# imprime a solução ótima
for v in pmr5.getVars():
    print(v.varName, v.x)

# imprime o valor ótimo
print('opt pmr5: ', pmr5.objVal)

# get valores do dual
#for c in pmr5.getConstrs():
#    print ('The dual value of %s : %g' %(c.constrName,c.pi))
    
mi1 = rest1.pi
mi2 = rest2.pi
mi3 = rest3.pi

print("mi1 :", mi1)
print("mi2 :", mi2)
print("mi3 :", mi3)

In [ ]:
# subproblema 5

subp5 = gp.Model()

# variáveis
x1 = subp5.addVar(obj=2 - 2*(mi1) - (mi2), name="x1")
x2 = subp5.addVar(obj=4 - (mi1) - (mi2), name="x2")
x3 = subp5.addVar(obj=1 - (mi1) + (mi2), name="x3")

# restrições
rest1 = subp5.addConstr(x1 <= 4, "restricao1")
rest2 = subp5.addConstr(x2 <= 6, "restricao2")
rest3 = subp5.addConstr(x3 <= 6, "restricao3")
rest4 = subp5.addConstr(x3 >= 1, "restricao4")

# definindo sentido da funcao objetivo
subp5.modelSense = GRB.MAXIMIZE

# escrevendo .lp/.mps
subp5.write('subp5.lp')

# resolve o problema
subp5.optimize()

# imprime a solução ótima
for v in subp5.getVars():
    print(v.varName, v.x)
    
# imprime o valor ótimo
print('opt subp5: ', subp5.objVal - (mi3))

In [ ]:
# limite dual
print('Limite dual: ',  pmr5.objVal + (subp5.objVal - mi3))

In [ ]:
x = v1*l1.X + v2*l2.X + v3*l3.X + v4*l4.X + v5*l5.X
print("x = ", x)

In [ ]:
c = np.array([2, 4, 1])
optimo = c@x
print("otimo :", optimo)